# Training with Contrastive Learning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
#from torchvision import datasets, models, transforms
from torchvision.models import resnet18,ResNet18_Weights
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import tqdm
from glob import glob
import random
from torchvision.io import read_image, ImageReadMode

Here I just get a ResNet18 model, and replace its end with a 256-dimensional output layer

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

model = resnet18(weights=ResNet18_Weights.DEFAULT)
num_feats = model.fc.in_features
model.fc = nn.Linear(num_feats, 256)

In [3]:
transform = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size=(200,200), antialias=True),
    v2.RandomRotation(30),
    v2.RandomPerspective(),
    v2.ToDtype(torch.float32, scale=True)
])

class ContLearnDataset(Dataset):
    def __init__(self, dir, transforms=None):
        self.dir = dir
        self.transforms = transforms
        self.filenames = glob(dir+'/*.jpg')

    def __getitem__(self, idx):
        a = read_image(self.filenames[idx], mode=ImageReadMode.RGB)
        p = a.detach().clone()
        i = torch.randint(len(self.filenames),(1,))[0]
        while i==idx:
            i = torch.randint(len(self.filenames),(1,))[0]
        n = read_image(self.filenames[i], mode=ImageReadMode.RGB)
        return self.transforms(a), self.transforms(p), self.transforms(n)

    def __len__(self):
        return len(self.filenames)

In [4]:
cld = ContLearnDataset('/home/scs/taylor/faculty/midPics',transforms=transform)

dl = DataLoader(cld, batch_size=64, num_workers=10)

In [5]:
model=model.to('cuda')
EPOCHS = 501

criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)

for epoch in tqdm.tqdm(range(EPOCHS)):
    totalloss=0
    for batch, (a, p, n) in enumerate(dl):
        a,p,n = a.to('cuda'), p.to('cuda'), n.to('cuda')
        aem = model(a)
        pem = model(p)
        nem = model(n)
        loss = criterion(aem, pem, nem)

            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        totalloss+=loss.item()
    if epoch%10==0:
        print(totalloss)
torch.save(model, 'fr.pt')

  0%|                                         | 1/501 [00:14<1:57:14, 14.07s/it]

27.044744189828634


  2%|▉                                       | 11/501 [02:32<1:53:17, 13.87s/it]

5.7630290780216455


  4%|█▋                                      | 21/501 [04:51<1:51:05, 13.89s/it]

...

0.30618688464164734


 98%|██████████████████████████████████████▏| 491/501 [1:54:19<02:18, 13.89s/it]

0.5570314861834049


100%|███████████████████████████████████████| 501/501 [1:56:38<00:00, 13.97s/it]

0.09460968151688576


In [7]:
model=torch.load('fr.pt')
model=model.to('cuda')

In [9]:
a,p,n = cld2[20]
a,p,n = a.reshape((1,3,112,92)), p.reshape((1,3,112,92)), n.reshape((1,3,112,92))
vals=torch.cat((a,p,n),dim=0).to('cuda')
res=model(vals).detach()
print(res.shape)
res[0].shape
print(f'a/p: {nn.functional.mse_loss(res[0],res[1])}')
print(f'a/n: {nn.functional.mse_loss(res[0],res[2])}')
print(f'p/n: {nn.functional.mse_loss(res[1],res[2])}')


torch.Size([3, 256])
a/p: 0.014126414433121681
a/n: 0.5725901126861572
p/n: 0.5608456134796143


In [29]:
gallery=[]
for fn in glob('/home/scs/taylor/faculty/onlySD312/*.jpg'):
    img = transform(read_image(fn, mode=ImageReadMode.RGB))
    gallery.append(img)

In [37]:
embeds=[]
for img in gallery:
    shape = img.shape
    img=img.reshape((1,shape[0],shape[1],shape[2]))
    embeds.append(model(img.to('cuda')).detach().cpu().numpy())

In [38]:
embeds = np.concatenate(embeds,axis=0)
embeds.shape

(39, 256)